# Integrating a production system with BugDoc

For BugDoc to work, every run of a particular configuration $CP_i$ should give the same success/failure result. That's our definition of determinism. Note that non-determinism can arise either through inputs controlled by random processes (e.g. random number generators) or through side effects. For example, if a database $D$ is the input to a computational pipeline and another pipeline or external process can change $D$, then running some instance $CP_i$ at one time may yield a different result than running $CP_i$ at another time, because D might have changed in between.

In order to link BugDoc with a working production system but without interfering with that production system, we need to perform several steps. We outline them below along with a running example.


## Creating a sandbox 


To achieve determinism for a pipeline configuration $CP_i$, we would had to snapshot the state of $D$ used as the input to $CP_i$. $CP_i$ should then run on that snapshot of $D$. We implemented snapshots by running each pipeline instance in a sandbox with independent file systems. The snapshots are parameters ($D$ in this example) that BugDoc can manipulate.


## Evaluating the pipeline

The pipeline needs to be rebuilt in the sandbox with an evaluation function attached
on the output to say whether a given execution through the pipeline
succeeded or failed.


## Execution History
We keep track of the provenance of each execution -- parameter settings,
data inputs, data outputs, and result of the evaluation function.

In [1]:
from bugdoc.utils.utils import record_pipeline_run

##  Satisfying BugDoc's API
the sandbox must receive a messag from BugDoc, run the pipeline and send
back success or failure

In [ ]:
"""
Worker script
===========================

This script responsible for receiving pipeline configurations from BugDoc's algorithms.
It runs and evaluates the pipeline instances, and returns the result to BugDoc.
"""

# %%
# Importing necessary packages.
# ------------------------
# We load utility packages to open communication and store pipeline instances.

import ast
import sys
import traceback
import zmq
from bugdoc.utils.utils import record_pipeline_run

# %%
# Importing pipeline engine API.
# ------------------------
# Here we load the functions that execute and evaluate a pipeline instance.
from my_api_example import execute_pipeline, evaluate_pipeline_output


host = 'localhost'
receive = '5557'
send = '5558'

context = zmq.Context()

# Socket to receive messages on
receiver = context.socket(zmq.PULL)
receiver.connect("tcp://{0}:{1}".format(host, receive))

# Socket to send messages to
sender = context.socket(zmq.PUSH)
sender.connect("tcp://{0}:{1}".format(host, send))


# Process tasks forever
while True:
    # Receive message from BugDoc
    data = receiver.recv_string()
    fields = data.split("|")
    filename = fields[0]
    values = ast.literal_eval(fields[1])
    parameters = ast.literal_eval(fields[2])
    try:
        # Recontruct pipeline configuration
        configuration = {
            parameters[i]: values[i]
            for i in range(len(parameters))
        }
        # Execute pipeline
        output = execute_pipeline(filename, configuration)
        # Evaluate pipeline
        result = evaluate_pipeline_output(output)
    except:
        traceback.print_exc(file=sys.stdout)
        # Assign failure in case of crash
        result = False
    # Record provenance
    record_pipeline_run(filename, values, parameters, result)
    values.append(result)
    # Send result back to BugDoc
    sender.send_string(str(values))
